In [57]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
import sys
import nltk

In [58]:
#loading the data
train_dataset = pd.read_json("Kickstarter_2022-12-15T03_20_04_521Z.json", lines=True)
actual_data = [train_dataset["data"][k] for k in range(len(train_dataset))]

In [59]:
#creating the dataset
train_data = pd.DataFrame(actual_data)
len(train_data)

7990

In [60]:
#exploring some of the features
print("The features of this dataset are: ", train_data.keys())
print(" \n \n blurb: ", train_data["blurb"][0], "\n \n ", "location: ", train_data["location"][0], "\n \n ", "category: ", train_data["category"][0], "\n \n ", "profile: ", train_data["profile"][0])

The features of this dataset are:  Index(['id', 'photo', 'name', 'blurb', 'goal', 'pledged', 'state', 'slug',
       'disable_communication', 'country', 'country_displayable_name',
       'currency', 'currency_symbol', 'currency_trailing_code', 'deadline',
       'state_changed_at', 'created_at', 'launched_at', 'staff_pick',
       'is_starrable', 'backers_count', 'static_usd_rate', 'usd_pledged',
       'converted_pledged_amount', 'fx_rate', 'usd_exchange_rate',
       'current_currency', 'usd_type', 'creator', 'location', 'category',
       'profile', 'spotlight', 'urls', 'source_url', 'friends', 'is_starred',
       'is_backing', 'permissions'],
      dtype='object')
 
 
 blurb:  Live camera installed inside an actual driving school 
 
  location:  {'id': 2363796, 'name': 'Beverly Hills', 'slug': 'beverly-hills-ca', 'short_name': 'Beverly Hills, CA', 'displayable_name': 'Beverly Hills, CA', 'localized_name': 'Beverly Hills', 'country': 'US', 'state': 'CA', 'type': 'Town', 'is_root':

In [61]:
#writing a function to create the final dataset
def clean_concat(list_of_datasets):
    data_frames=[]
    for dataset in list_of_datasets:
        dataset = pd.read_json(dataset, lines=True)
        actual_data = [dataset["data"][k] for k in range(len(dataset))]
        data_frames.append(pd.DataFrame(actual_data))
    final_dataset = pd.concat([data for data in data_frames])
    return(final_dataset)

In [62]:
# concatenating creates problems with the indexes 
#training_df = clean_concat(["/Users/ghitaalami/Coding Projects/Berkeley Projects/ML-and-Data-Analytics/Project/Kickstarter_2022-12-15T03_20_04_521Z.json", "/Users/ghitaalami/Coding Projects/Berkeley Projects/ML-and-Data-Analytics/Project/Kickstarter_2023-01-05T03_20_04_136Z.json"])

In [63]:
training_df = train_data

In [64]:
#let's decide which columns should be dropped
training_df["photo"][0] #should probably be dropped for now
training_df["state"][0] #our label 
training_df["urls"][0] #should probably be dropped 
training_df["currency"][10] #should be dropped because "currency_symbol" contains the same information
training_df["source_url"][200] #should probably be dropped for now 
training_df["friends"][1600] #should check if only contains NaNs
training_df["permissions"][1000] #should check if only contains NaNs



nan

In [65]:
#modify the clean_concat function to drop some columns
def clean_concat_drop(list_of_datasets, list_of_columns):
    data_frames=[]
    for dataset in list_of_datasets:
        dataset = pd.read_json(dataset, lines=True)
        actual_data = [dataset["data"][k] for k in range(len(dataset))]
        data_frames.append(pd.DataFrame(actual_data))
    final_dataset = pd.concat([data for data in data_frames])
    final_dataset = final_dataset.drop(list_of_columns)
    return(final_dataset)

In [66]:
#dropping some useless features for now
training_df = training_df.drop(["photo", "urls", "currency", "source_url" ], axis=1)

In [67]:
training_df.head(5)

,id,name,blurb,goal,pledged,state,slug,disable_communication,country,country_displayable_name,...,usd_type,creator,location,category,profile,spotlight,friends,is_starred,is_backing,permissions
0,2014941593,Driving School Stories of Celebrity Teens,Live camera installed inside an actual driving...,120000,26.0,failed,driving-school-stories-of-celebrity-teens,False,US,the United States,...,international,"{'id': 1097743720, 'name': 'Kamran Rafizadeh F...","{'id': 2363796, 'name': 'Beverly Hills', 'slug...","{'id': 292, 'name': 'Comedy', 'analytics_name'...","{'id': 4465571, 'project_id': 4465571, 'state'...",False,NaN,NaN,NaN,NaN
1,1932477189,Crime Squad 3D Blu-ray - REMASTER3D!,Crime Squad 3D in Superstereovision! Remastere...,2000,2541.0,live,crime-squad-3d-blu-ray,False,GB,the United Kingdom,...,international,"{'id': 115821324, 'name': 'Andrew Murchie', 's...","{'id': 19344, 'name': 'Edinburgh', 'slug': 'ed...","{'id': 292, 'name': 'Comedy', 'analytics_name'...","{'id': 4516474, 'project_id': 4516474, 'state'...",False,NaN,NaN,NaN,NaN
2,1127603300,Help Us Get To Los Angeles!,"As winners of the 48-Hour Film Project, we hop...",16000,106.0,live,help-subharmonic-city-get-to-hollywood,False,US,the United States,...,international,"{'id': 475346952, 'name': 'Subharmonic City Pr...","{'id': 2397816, 'name': 'El Paso', 'slug': 'el...","{'id': 292, 'name': 'Comedy', 'analytics_name'...","{'id': 4518603, 'project_id': 4518603, 'state'...",False,NaN,NaN,NaN,NaN
3,640015337,SECRET,A Superhero Comedy,50000,501.0,live,secret-0,False,US,the United States,...,international,"{'id': 66032665, 'name': 'Brian Cutter', 'slug...","{'id': 2416847, 'name': 'Hampton', 'slug': 'ha...","{'id': 292, 'name': 'Comedy', 'analytics_name'...","{'id': 4521089, 'project_id': 4521089, 'state'...",False,NaN,NaN,NaN,NaN
4,233782977,PINK (Short Film),A neurotic actress falls into crisis at her bi...,2000,1176.0,live,pink-short-film,False,US,the United States,...,international,"{'id': 1827951120, 'name': 'Jackson Cook', 'sl...","{'id': 2379574, 'name': 'Chicago', 'slug': 'ch...","{'id': 292, 'name': 'Comedy', 'analytics_name'...","{'id': 4519929, 'project_id': 4519929, 'state'...",False,NaN,NaN,NaN,NaN


In [68]:
#Let's try a first model with less features
training_df = training_df.drop("country_displayable_name", axis=1)
training_df = training_df.drop(["is_backing", "is_starred", "spotlight", "friends", "permissions", "disable_communication"], axis=1)

In [69]:
training_df.head(5)

,id,name,blurb,goal,pledged,state,slug,country,currency_symbol,currency_trailing_code,...,usd_pledged,converted_pledged_amount,fx_rate,usd_exchange_rate,current_currency,usd_type,creator,location,category,profile
0,2014941593,Driving School Stories of Celebrity Teens,Live camera installed inside an actual driving...,120000,26.0,failed,driving-school-stories-of-celebrity-teens,US,$,True,...,26.0,26.0,1.000000,1.000000,USD,international,"{'id': 1097743720, 'name': 'Kamran Rafizadeh F...","{'id': 2363796, 'name': 'Beverly Hills', 'slug...","{'id': 292, 'name': 'Comedy', 'analytics_name'...","{'id': 4465571, 'project_id': 4465571, 'state'..."
1,1932477189,Crime Squad 3D Blu-ray - REMASTER3D!,Crime Squad 3D in Superstereovision! Remastere...,2000,2541.0,live,crime-squad-3d-blu-ray,GB,£,False,...,3082.93718733,3137.0,1.234778,1.234778,USD,international,"{'id': 115821324, 'name': 'Andrew Murchie', 's...","{'id': 19344, 'name': 'Edinburgh', 'slug': 'ed...","{'id': 292, 'name': 'Comedy', 'analytics_name'...","{'id': 4516474, 'project_id': 4516474, 'state'..."
2,1127603300,Help Us Get To Los Angeles!,"As winners of the 48-Hour Film Project, we hop...",16000,106.0,live,help-subharmonic-city-get-to-hollywood,US,$,True,...,106.0,106.0,1.000000,1.000000,USD,international,"{'id': 475346952, 'name': 'Subharmonic City Pr...","{'id': 2397816, 'name': 'El Paso', 'slug': 'el...","{'id': 292, 'name': 'Comedy', 'analytics_name'...","{'id': 4518603, 'project_id': 4518603, 'state'..."
3,640015337,SECRET,A Superhero Comedy,50000,501.0,live,secret-0,US,$,True,...,501.0,501.0,1.000000,1.000000,USD,international,"{'id': 66032665, 'name': 'Brian Cutter', 'slug...","{'id': 2416847, 'name': 'Hampton', 'slug': 'ha...","{'id': 292, 'name': 'Comedy', 'analytics_name'...","{'id': 4521089, 'project_id': 4521089, 'state'..."
4,233782977,PINK (Short Film),A neurotic actress falls into crisis at her bi...,2000,1176.0,live,pink-short-film,US,$,True,...,1176.0,1176.0,1.000000,1.000000,USD,international,"{'id': 1827951120, 'name': 'Jackson Cook', 'sl...","{'id': 2379574, 'name': 'Chicago', 'slug': 'ch...","{'id': 292, 'name': 'Comedy', 'analytics_name'...","{'id': 4519929, 'project_id': 4519929, 'state'..."


In [90]:
#only keeping the name of the city in the location
training_df["location"] = [training_df["location"][k]["name"] if type(training_df["location"][k]) is dict else 'nan' for k in range(len(training_df)) ]
training_df.head(5)

,id,name,blurb,goal,pledged,state,slug,country,currency_symbol,currency_trailing_code,...,usd_pledged,converted_pledged_amount,fx_rate,usd_exchange_rate,current_currency,usd_type,creator,location,category,profile
0,2014941593,Driving School Stories of Celebrity Teens,Live camera installed inside an actual driving...,120000,26.0,failed,driving-school-stories-of-celebrity-teens,US,$,True,...,26.0,26.0,1.000000,1.000000,USD,international,"{'id': 1097743720, 'name': 'Kamran Rafizadeh F...",Beverly Hills,"{'id': 292, 'name': 'Comedy', 'analytics_name'...","{'id': 4465571, 'project_id': 4465571, 'state'..."
1,1932477189,Crime Squad 3D Blu-ray - REMASTER3D!,Crime Squad 3D in Superstereovision! Remastere...,2000,2541.0,live,crime-squad-3d-blu-ray,GB,£,False,...,3082.93718733,3137.0,1.234778,1.234778,USD,international,"{'id': 115821324, 'name': 'Andrew Murchie', 's...",Edinburgh,"{'id': 292, 'name': 'Comedy', 'analytics_name'...","{'id': 4516474, 'project_id': 4516474, 'state'..."
2,1127603300,Help Us Get To Los Angeles!,"As winners of the 48-Hour Film Project, we hop...",16000,106.0,live,help-subharmonic-city-get-to-hollywood,US,$,True,...,106.0,106.0,1.000000,1.000000,USD,international,"{'id': 475346952, 'name': 'Subharmonic City Pr...",El Paso,"{'id': 292, 'name': 'Comedy', 'analytics_name'...","{'id': 4518603, 'project_id': 4518603, 'state'..."
3,640015337,SECRET,A Superhero Comedy,50000,501.0,live,secret-0,US,$,True,...,501.0,501.0,1.000000,1.000000,USD,international,"{'id': 66032665, 'name': 'Brian Cutter', 'slug...",Hampton,"{'id': 292, 'name': 'Comedy', 'analytics_name'...","{'id': 4521089, 'project_id': 4521089, 'state'..."
4,233782977,PINK (Short Film),A neurotic actress falls into crisis at her bi...,2000,1176.0,live,pink-short-film,US,$,True,...,1176.0,1176.0,1.000000,1.000000,USD,international,"{'id': 1827951120, 'name': 'Jackson Cook', 'sl...",Chicago,"{'id': 292, 'name': 'Comedy', 'analytics_name'...","{'id': 4519929, 'project_id': 4519929, 'state'..."


In [ ]:
# next steps : only keeping some elements of the dictionaries for features such as creator, profile, category
# perform lemmatization, stemming, remove_stopwords, tokenization on the texts remaining
# gather all the texts in one column ({location: 'Paris' , category: 'comedy'} become {text: 'location_Paris, category_comedy'} for example)
# fit a classification model on the resulting dataset